# Spark SQL基础

本笔记本介绍Spark SQL，这是Spark用于处理结构化数据的模块。Spark SQL允许您使用SQL语句查询数据，同时还提供了与其他Spark组件的无缝集成。

## 什么是Spark SQL？

Spark SQL是Spark的一个模块，用于处理结构化数据。它提供了一个编程抽象，称为DataFrame，并且可以作为分布式SQL查询引擎。

Spark SQL的主要特点：
- **集成**：可以在SQL和编程API之间无缝切换
- **统一数据访问**：可以连接到各种数据源
- **兼容Hive**：可以运行未修改的Hive查询
- **标准连接**：支持JDBC/ODBC连接

## 1. 创建SparkSession

SparkSession是Spark SQL的入口点，它允许您创建DataFrame和执行SQL查询。

In [ ]:
from pyspark.sql import SparkSession

# 创建SparkSession
spark = SparkSession.builder \
    .appName("Spark SQL基础") \
    .getOrCreate()

# 显示Spark版本
print(f"Spark版本: {spark.version}")

## 2. 加载数据

Spark SQL可以从各种数据源加载数据，包括CSV、JSON、Parquet等。

In [ ]:
# 加载CSV数据
sales_df = spark.read.option("header", "true") \
                    .option("inferSchema", "true") \
                    .csv("/home/jovyan/data/sample/sales_data.csv")

# 显示数据
print("销售数据:")
sales_df.show()

# 显示Schema
print("\n销售数据Schema:")
sales_df.printSchema()

In [ ]:
# 加载JSON数据
user_df = spark.read.json("/home/jovyan/data/sample/user_behavior.json")

print("用户行为数据:")
user_df.show()

# 显示Schema
print("\n用户行为数据Schema:")
user_df.printSchema()

## 3. 创建临时视图

要使用SQL查询DataFrame，首先需要将其注册为临时视图。

In [ ]:
# 注册临时视图
sales_df.createOrReplaceTempView("sales")
user_df.createOrReplaceTempView("user_behavior")

# 创建全局临时视图（跨SparkSession可用）
sales_df.createGlobalTempView("global_sales")

## 4. 基本SQL查询

现在，我们可以使用SQL语句查询数据。

In [ ]:
# 简单的SELECT查询
result = spark.sql("""
    SELECT * FROM sales LIMIT 5
""")

print("简单查询结果:")
result.show()

In [ ]:
# 选择特定列
result = spark.sql("""
    SELECT product_id, category, price FROM sales
""")

print("选择特定列:")
result.show(5)

In [ ]:
# 使用WHERE子句过滤数据
result = spark.sql("""
    SELECT * FROM sales
    WHERE price > 100
""")

print("价格大于100的商品:")
result.show()

## 5. 聚合和分组

SQL的强大之处在于其聚合和分组功能。

In [ ]:
# 按类别分组并计算总销售额
result = spark.sql("""
    SELECT category, SUM(price * quantity) AS total_sales
    FROM sales
    GROUP BY category
    ORDER BY total_sales DESC
""")

print("按类别统计销售额:")
result.show()

In [ ]:
# 计算多个聚合
result = spark.sql("""
    SELECT 
        category,
        COUNT(*) AS count,
        SUM(price) AS total_price,
        AVG(price) AS avg_price,
        MAX(price) AS max_price,
        MIN(price) AS min_price
    FROM sales
    GROUP BY category
""")

print("按类别的详细统计:")
result.show()

In [ ]:
# 使用HAVING子句过滤分组结果
result = spark.sql("""
    SELECT category, COUNT(*) AS count
    FROM sales
    GROUP BY category
    HAVING COUNT(*) > 2
""")

print("商品数量大于2的类别:")
result.show()

## 6. 连接操作

SQL连接操作允许您组合多个表的数据。

In [ ]:
# 创建客户数据
customers_data = [
    ("C1001", "张三", "北京"),
    ("C1002", "李四", "上海"),
    ("C1003", "王五", "广州"),
    ("C1004", "赵六", "深圳"),
    ("C1005", "钱七", "杭州"),
    ("C1006", "孙八", "成都"),
    ("C1007", "周九", "武汉")
]
customers_df = spark.createDataFrame(customers_data, ["customer_id", "name", "city"])
customers_df.createOrReplaceTempView("customers")

# 显示客户数据
print("客户数据:")
customers_df.show()

In [ ]:
# 内连接
result = spark.sql("""
    SELECT s.date, c.name, s.product_id, s.price, s.quantity
    FROM sales s
    JOIN customers c ON s.customer_id = c.customer_id
""")

print("销售和客户数据内连接:")
result.show()

In [ ]:
# 左外连接
result = spark.sql("""
    SELECT c.name, c.city, s.product_id, s.price
    FROM customers c
    LEFT OUTER JOIN sales s ON c.customer_id = s.customer_id
""")

print("左外连接结果:")
result.show()

In [ ]:
# 右外连接
result = spark.sql("""
    SELECT c.name, c.city, s.product_id, s.price
    FROM customers c
    RIGHT OUTER JOIN sales s ON c.customer_id = s.customer_id
""")

print("右外连接结果:")
result.show()

In [ ]:
# 全外连接
result = spark.sql("""
    SELECT c.name, c.city, s.product_id, s.price
    FROM customers c
    FULL OUTER JOIN sales s ON c.customer_id = s.customer_id
""")

print("全外连接结果:")
result.show()

## 7. 子查询和公共表表达式(CTE)

子查询和CTE可以帮助您构建复杂的查询。

In [ ]:
# 使用子查询
result = spark.sql("""
    SELECT * FROM (
        SELECT category, SUM(price * quantity) AS total_sales
        FROM sales
        GROUP BY category
    ) AS category_sales
    WHERE total_sales > 1000
""")

print("使用子查询:")
result.show()

In [ ]:
# 使用公共表表达式(CTE)
result = spark.sql("""
    WITH category_sales AS (
        SELECT category, SUM(price * quantity) AS total_sales
        FROM sales
        GROUP BY category
    ),
    region_sales AS (
        SELECT region, SUM(price * quantity) AS total_sales
        FROM sales
        GROUP BY region
    )
    SELECT c.category, c.total_sales AS category_sales, r.region, r.total_sales AS region_sales
    FROM category_sales c
    CROSS JOIN region_sales r
    ORDER BY c.total_sales DESC, r.total_sales DESC
""")

print("使用公共表表达式:")
result.show()

## 8. 窗口函数

窗口函数允许您在分组数据上执行计算，而不会将结果合并为单个输出行。

In [ ]:
# 使用窗口函数计算每个类别中的排名
result = spark.sql("""
    SELECT 
        category,
        product_id,
        price,
        RANK() OVER (PARTITION BY category ORDER BY price DESC) AS price_rank
    FROM sales
""")

print("使用窗口函数计算排名:")
result.show()

In [ ]:
# 计算累计总和
result = spark.sql("""
    SELECT 
        date,
        product_id,
        price,
        SUM(price) OVER (ORDER BY date) AS cumulative_price
    FROM sales
""")

print("计算累计总和:")
result.show()

In [ ]:
# 计算移动平均
result = spark.sql("""
    SELECT 
        date,
        product_id,
        price,
        AVG(price) OVER (
            ORDER BY date 
            ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
        ) AS moving_avg_price
    FROM sales
""")

print("计算移动平均:")
result.show()

## 9. 用户自定义函数(UDF)

UDF允许您在SQL查询中使用自定义函数。

In [ ]:
# 定义UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# 定义一个函数，将价格分类为"低"、"中"或"高"
def price_category(price):
    if price < 50:
        return "低"
    elif price < 200:
        return "中"
    else:
        return "高"

# 注册UDF
spark.udf.register("price_category", price_category, StringType())

# 在SQL查询中使用UDF
result = spark.sql("""
    SELECT 
        product_id,
        price,
        price_category(price) AS price_category
    FROM sales
    ORDER BY price
""")

print("使用UDF分类价格:")
result.show()

## 10. 保存查询结果

您可以将SQL查询结果保存为各种格式。

In [ ]:
# 执行查询
category_sales = spark.sql("""
    SELECT category, SUM(price * quantity) AS total_sales
    FROM sales
    GROUP BY category
    ORDER BY total_sales DESC
""")

# 保存为CSV
category_sales.write.mode("overwrite").option("header", "true").csv("/home/jovyan/data/output/category_sales_csv")

# 保存为Parquet
category_sales.write.mode("overwrite").parquet("/home/jovyan/data/output/category_sales_parquet")

# 保存为JSON
category_sales.write.mode("overwrite").json("/home/jovyan/data/output/category_sales_json")

## 11. 实际案例：销售数据分析

让我们使用SQL进行一些更复杂的销售数据分析。

In [ ]:
# 1. 按地区和类别的销售额分析
result = spark.sql("""
    SELECT 
        region,
        category,
        SUM(price * quantity) AS total_sales,
        COUNT(DISTINCT customer_id) AS customer_count,
        SUM(price * quantity) / COUNT(DISTINCT customer_id) AS sales_per_customer
    FROM sales
    GROUP BY region, category
    ORDER BY region, total_sales DESC
""")

print("按地区和类别的销售分析:")
result.show()

In [ ]:
# 2. 找出每个类别中销售额最高的产品
result = spark.sql("""
    WITH product_sales AS (
        SELECT 
            category,
            product_id,
            SUM(price * quantity) AS total_sales,
            RANK() OVER (PARTITION BY category ORDER BY SUM(price * quantity) DESC) AS sales_rank
        FROM sales
        GROUP BY category, product_id
    )
    SELECT 
        category,
        product_id,
        total_sales
    FROM product_sales
    WHERE sales_rank = 1
    ORDER BY total_sales DESC
""")

print("每个类别中销售额最高的产品:")
result.show()

In [ ]:
# 3. 客户购买行为分析
result = spark.sql("""
    WITH customer_purchases AS (
        SELECT 
            c.name,
            c.city,
            COUNT(*) AS purchase_count,
            SUM(s.price * s.quantity) AS total_spent,
            AVG(s.price) AS avg_price,
            MAX(s.price) AS max_price
        FROM sales s
        JOIN customers c ON s.customer_id = c.customer_id
        GROUP BY c.name, c.city
    )
    SELECT 
        name,
        city,
        purchase_count,
        total_spent,
        avg_price,
        max_price,
        CASE 
            WHEN total_spent > 1000 THEN '高价值'
            WHEN total_spent > 500 THEN '中价值'
            ELSE '低价值'
        END AS customer_value
    FROM customer_purchases
    ORDER BY total_spent DESC
""")

print("客户购买行为分析:")
result.show()

## 12. 练习

现在，让我们通过一些练习来巩固所学知识。

### 练习1：用户行为分析

使用用户行为数据，编写SQL查询完成以下任务：
1. 统计每种行为（view、add_to_cart、purchase）的次数
2. 找出浏览次数最多的商品
3. 计算每个用户的购买转化率（购买次数/浏览次数）

In [ ]:
# 1. 统计每种行为的次数
result = spark.sql("""
    SELECT action, COUNT(*) AS count
    FROM user_behavior
    GROUP BY action
    ORDER BY count DESC
""")

print("各类行为次数:")
result.show()

In [ ]:
# 2. 找出浏览次数最多的商品
result = spark.sql("""
    SELECT item_id, COUNT(*) AS view_count
    FROM user_behavior
    WHERE action = 'view'
    GROUP BY item_id
    ORDER BY view_count DESC
    LIMIT 5
""")

print("浏览次数最多的商品:")
result.show()

In [ ]:
# 3. 计算每个用户的购买转化率
result = spark.sql("""
    WITH user_actions AS (
        SELECT 
            user_id,
            SUM(CASE WHEN action = 'view' THEN 1 ELSE 0 END) AS view_count,
            SUM(CASE WHEN action = 'purchase' THEN 1 ELSE 0 END) AS purchase_count
        FROM user_behavior
        GROUP BY user_id
    )
    SELECT 
        user_id,
        view_count,
        purchase_count,
        CASE 
            WHEN view_count > 0 THEN ROUND(purchase_count / view_count, 2)
            ELSE 0
        END AS conversion_rate
    FROM user_actions
    WHERE view_count > 0
    ORDER BY conversion_rate DESC
""")

print("用户购买转化率:")
result.show()

### 练习2：销售数据高级分析

使用销售数据，编写SQL查询完成以下任务：
1. 按月份统计销售趋势
2. 计算每个类别的销售占比
3. 找出购买频率最高的客户

In [ ]:
# 1. 按月份统计销售趋势
result = spark.sql("""
    SELECT 
        SUBSTRING(date, 1, 7) AS month,
        SUM(price * quantity) AS total_sales
    FROM sales
    GROUP BY SUBSTRING(date, 1, 7)
    ORDER BY month
""")

print("月度销售趋势:")
result.show()

In [ ]:
# 2. 计算每个类别的销售占比
result = spark.sql("""
    WITH category_sales AS (
        SELECT 
            category,
            SUM(price * quantity) AS category_sales
        FROM sales
        GROUP BY category
    ),
    total AS (
        SELECT SUM(price * quantity) AS total_sales
        FROM sales
    )
    SELECT 
        c.category,
        c.category_sales,
        t.total_sales,
        ROUND(c.category_sales / t.total_sales * 100, 2) AS sales_percentage
    FROM category_sales c, total t
    ORDER BY sales_percentage DESC
""")

print("类别销售占比:")
result.show()

In [ ]:
# 3. 找出购买频率最高的客户
result = spark.sql("""
    SELECT 
        s.customer_id,
        c.name,
        COUNT(*) AS purchase_count,
        SUM(s.price * s.quantity) AS total_spent
    FROM sales s
    JOIN customers c ON s.customer_id = c.customer_id
    GROUP BY s.customer_id, c.name
    ORDER BY purchase_count DESC
    LIMIT 5
""")

print("客户购买频率排行:")
result.show()

## 13. 总结

在本笔记本中，我们学习了：

1. Spark SQL的基本概念和功能
2. 如何加载数据并创建临时视图
3. 基本SQL查询（SELECT、WHERE、GROUP BY等）
4. 高级SQL功能（连接、子查询、CTE、窗口函数）
5. 如何使用用户自定义函数(UDF)
6. 如何保存查询结果
7. 如何应用SQL解决实际问题

Spark SQL是一个强大的工具，它结合了SQL的易用性和Spark的分布式计算能力，使得大规模数据分析变得简单高效。

## 下一步

接下来，我们将学习更高级的SQL查询技术和优化策略。请继续学习 `advanced-queries.ipynb` 笔记本。

In [ ]:
# 停止SparkSession
spark.stop()